In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import numpy as np
import pygritbx

In [ ]:
# Unit Vectors for defining reference frame
i = np.array([1, 0, 0])
j = np.array([0, 1, 0])
k = np.array([0, 0, 1])

In [ ]:
# Axial dimensions on Shaft A1
a1 = 9 + 28.75 + 9 # [mm]
b1 = (13.25 - 9) + 24.25 + 30 / 2 # [mm]
zR1 = a1 + b1 # [mm]

In [ ]:
# Axial dimension on shaft A1'
zR2 = 0 # [mm]
zR3 = 55 # [mm]

In [ ]:
# Axial dimension on shaft A2
zR4 = 17 + 18 / 2 # [mm]
a2 = (46 - zR4) + 9.25 # [mm]
b2 = 16 + 25 + 16 # [mm]
c2 = (58.5 - 16) + 35 # [mm]


In [ ]:
# Input
motor = pygritbx.Motor(name="M1", loc=0, power=6.25e3, n=2000, axis=k)

In [ ]:
# Material
shaftMaterial = pygritbx.Material(name='34NiCrMo6', sigma_u=1050, sigma_y=950, sigma_Dm1=520, HB=0)
R1R4_material = pygritbx.Material(name='Steel', sigma_u=1060, sigma_y=950, sigma_Dm1=0, HB=335)
R2R3_material = pygritbx.Material(name='Steel', sigma_u=1950, sigma_y=1400, sigma_Dm1=0, HB=560)

In [ ]:
# Gears
# R1
R1 = pygritbx.Gear(name="R1", axis=k, loc=zR1, m_n=2.5, z=18, psi=20, phi_n=20, Q_v=7, FW=30, material=R1R4_material)

# R2
R2 = pygritbx.Gear(name="R2", axis=k, loc=zR2, m_n=2.5, z=73, psi=-20, phi_n=20, Q_v=7, FW=30, material=R2R3_material)

# R3
R3 = pygritbx.Gear(name="R3", axis=k, loc=zR3, m_n=2.5, z=19, psi=-20, phi_n=20, Q_v=7, FW=30, material=R2R3_material)

# R4
R4 = pygritbx.Gear(name="R4", axis=k, loc=zR4, m_n=2.5, z=51, psi=20, phi_n=20, Q_v=7, FW=30, material=R1R4_material)

In [ ]:
# Supports
# A (roller 30203)
A = pygritbx.Support(name="A", type="Roller", bearingType="Tapered", catalogueName="30203", catalogueType="Explorer",
                     d=17, D=40, B=13.25, C=23.4e3, C0=18.6e3, Pu=1.83e3, nr=15e3, a=9, e=0.35, X=0, Y=1.7, Y0=0.9, Y1=0,
                     Y2=0, A=0, kr=0, shoulder=-1, arr="B2B", axis=k, loc=0)

# B (roller 30203)
B = pygritbx.Support(name="B", type="Pin", bearingType="Tapered", catalogueName="30203", catalogueType="Explorer",
                     d=17, D=40, B=13.25, C=23.4e3, C0=18.6e3, Pu=1.83e3, nr=15e3, a=9, e=0.35, X=0, Y=1.7,Y0=0.9, Y1=0,
                     Y2=0, A=0, kr=0, shoulder=1, arr="B2B", axis=k, loc=a1)

# C (roller 30208)
C = pygritbx.Support(name="C", type="Roller", bearingType="Tapered", catalogueName="30208", catalogueType="Explorer",
                     d=40, D=80, B=19.75, C=75.8e3, C0=68e3, Pu=7.65e3, nr=7e3, a=16, e=0.37, X=0, Y=1.6, Y0=0.9, Y1=0,
                     Y2=0, A=0, kr=0, shoulder=-1, arr="B2B", axis=k, loc=zR4 + a2)

# D (roller 30208)
D = pygritbx.Support(name="D", type="Pin", bearingType="Tapered", catalogueName="30208", catalogueType="Explorer",
                     d=40, D=80, B=19.75, C=75.8e3, C0=68e3, Pu=7.65e3, nr=7e3, a=16, e=0.37, X=0, Y=1.6, Y0=0.9, Y1=0,
                     Y2=0, A=0, kr=0, shoulder=1, arr="B2B", axis=k, loc=zR4 + a2 + b2)

# F
F = pygritbx.Support(axis=k)

In [ ]:
# Output Component
output = pygritbx.Component("Output", None, k, zR4 + a2 + b2 + c2)

In [ ]:
# Gearbox Configuration
# Shaft A1
A1 = pygritbx.Shaft(name="A1", inputs=[motor], outputs=[R1], axis=k, material=shaftMaterial, sups=np.array([A, B]), loc=[0, 0, 0])
# Mesh M1
M1 = pygritbx.GearMesh(name="M1", drivingGear=R1, drivenGear=R2, axis=-j, type="External")
# Shaft A1'
A1_p = pygritbx.Shaft("A1'", [R2], [R3], -k, shaftMaterial, np.array([F, F]))
# Mesh M2
M2 = pygritbx.GearMesh("M2", R3, R4, j, "External")
# Shaft A2
A2 = pygritbx.Shaft("A2", [R4], [output], k, shaftMaterial, np.array([C, D]))

In [ ]:
A1.solve()

In [ ]:
R1.EFs[0].force, R1.ETs[0].torque

In [ ]:
A.F_tot.force, B.F_tot.force

In [ ]:
R2.solve()

In [ ]:
A1_p.solve()

In [ ]:
R3.solve()

In [ ]:
R4.solve()

In [ ]:
Fout_r = np.abs(np.sum(R4.ETs[0].torque)) / 0.1 * j
Fout_a = 0.25 * np.abs(np.sum(Fout_r)) * (-k)
Fout = pygritbx.Force(Fout_a + Fout_r, output.abs_loc)
A2.outputs[0].updateEFs([Fout])

In [ ]:
A2.solve()

In [ ]:
C.F_tot.force, D.F_tot.force

In [ ]:
# Shaft A2: profile for static verification
statProfile = pygritbx.ShaftProfile(np.array([34, 35, 35, 30, 30, 35, 35, 40, 40, 34, 34, 32, 32, 27, 27, 32, 32, 31]) / 2, 
                                  np.array([0, 1, 11.9, 12, 39.9, 40, 45.9, 46, 132.9, 133, 158.4, 158.5, 178.4, 178.5, 208.4, 208.5, 222, 223]))
statProfile.AddFillet(1, [2], 45, 18.5)
statProfile.AddFillet(1, [1], 134, 18)
statProfile_refined = statProfile.refineProfile(0.1)
A2.setProfile(statProfile_refined)
A2.profile.CalculateSectionProperties()

In [ ]:
# Shaft A2: define sections
# v1
v1 = pygritbx.ShaftSection('V1', np.array([0, 0, 26]), 30, 0.8, shaftMaterial)
v1.AppendKf(np.array([1.6, 2]), np.array(["Bending", "Torsion"]))


In [ ]:
# v2
v2 = pygritbx.ShaftSection('V2', np.array([0, 0, 46]), 40, 0.8, shaftMaterial)
v2.AddNotchSensitivity(1, A2.material.sigma_u)
v2.AddGeometricStressRaiser(1/35, 40/35)

In [ ]:
# v3
v3 = pygritbx.ShaftSection('V3', np.array([0, 0, 133]), 40, 0.8, shaftMaterial)
v3.AddNotchSensitivity(1, A2.material.sigma_u)
v3.AddGeometricStressRaiser(1/34, 40/34)

In [ ]:
userSections = np.array([v1, v2, v3])
A2.addSections(userSections)

In [ ]:
# Internal loads
A2.calculateInternalLoads(np.array([i, j, k]))

In [ ]:
# Plot internal loads
A2.plotInternalLoads()

In [ ]:
# Calculate stresses
A2.calculateStresses()
A2.calculateEquivalentAndIdealStress()

In [ ]:
A2.plotStresses()

In [ ]:
# Static safety factors
A2.calculateStaticSafetyFactor(sections=userSections)

In [ ]:
userSections[0].staticSF, userSections[1].staticSF, userSections[2].staticSF

In [ ]:
# Shaft A2: profile for fatigue verification
fatigueProfile = pygritbx.ShaftProfile(np.array([34, 35, 35, 40, 40, 34, 34, 32, 32, 31]) / 2,
                                     np.array([0, 1, 45.9, 46, 132.9, 133, 158.4, 158.5, 222, 223]))
fatigueProfile.AddFillet(1, [2], 45, 18.5)
fatigueProfile.AddFillet(1, [1], 134, 18)
fatigueProfile_refined = fatigueProfile.refineProfile(0.1)
A2.setProfile(fatigueProfile_refined)
A2.profile.CalculateSectionProperties()
A2.sections[0].d = 35
userSections[0].d = 35

In [ ]:
# Recalculate internal loads and stresses
A2.calculateInternalLoads(np.array([i, j, k]))
A2.calculateStresses()

In [ ]:
# Recalculate total and equivalent stress
A2.calculateEquivalentAndIdealStress()

In [ ]:
# replot internal loads and stress
A2.plotInternalLoads()

In [ ]:
# Replot stresses
A2.plotStresses()

In [ ]:
# Calculate mean and alternating stresses
A2.calculateMeanAlternatingStress(userSections)

In [ ]:
# Fatigue limit corrector factor
A2.insertFLCF(userSections)

In [ ]:
# Equivalent mean and alternating stress
A2.calculateEquivalentStresses(userSections)

In [ ]:
# Plot haigh diagram
for section in userSections:
    section.PlotHaighDiagram()

In [ ]:
# Fatigue sagety factor
for section in A2.sections:
    section.CalculateFatigueSF()
    print(f"{section.name}: {section.fatigueSF}")

In [ ]:
R1.analyseGearToothBending(mesh=M1, powerSource="Uniform", drivenMachine="Uniform", dShaft=35, Ce=1, teethCond="uncrowned teeth",
                           lShaft=a1+b1+50, useCond="Commercial, enclosed units", sigma_FP=320, b_YN=1.3558, e_YN=-0.0178,
                           N=1e8, temp=60, rel=0.99)

In [ ]:
# Maximum tooth gear bending stress
R1.calculateSigmaMaxFatigue(M1, "Uniform", "Uniform", 35, 1, "uncrowned teeth", a1 + b1 + 50, "Commercial, enclosed units")
R3.calculateSigmaMaxFatigue(M2, "Uniform", "Uniform", 30, 1, "uncrowned teeth", 70, "Commercial, enclosed units")

In [ ]:
R1.sigma_max_fatigue, R3.sigma_max_fatigue

In [ ]:
# Bending safety factor
R1.calculateBendingSF(320, 1.3558, -0.0178, 1e8, 60, 0.99)
R3.calculateBendingSF(450, 1.3558, -0.0178, 1e8, 60, 0.99)

In [ ]:
R1.bendingSF, R3.bendingSF

In [ ]:
# Maximum gear contact (pitting resistance) stress
R1.calculateSigmaMaxPitting(M1, 1)
R3.calculateSigmaMaxPitting(M2, 1)

In [ ]:
R1.sigma_max_pitting, R3.sigma_max_pitting

In [ ]:
# Wear safety factor
R1.calculateWearSF(860, 1.4488, -0.023, 1e8, M1)
R3.calculateWearSF(1360, 1.4488, -0.023, 1e8, M2)

In [ ]:
R1.wearSF, R3.wearSF

In [ ]:
A.performLifeAnalysis(rel=95, condition="Slight contamination", a_skf=2)

In [ ]:
B.performLifeAnalysis(rel=95, condition="Slight contamination", a_skf=5)

In [ ]:
C.performLifeAnalysis(rel=95, condition="Slight contamination", a_skf=1)

In [ ]:
D.performLifeAnalysis(rel=95, condition="Slight contamination", a_skf=2)

In [ ]:
# Define oil of choice
oil = pygritbx.Oil("ISO VG 100", 60, 15, 45)